In [222]:
def dict_factory(cursor, row):
    fields = [column[0] for column in cursor.description]
    return {key: value for key, value in zip(fields, row)}

In [223]:
import sqlite3
import pandas as pd

con = sqlite3.connect("superstore3.db")
con.row_factory = dict_factory
cur = con.cursor()

In [224]:
def run(sql):
    r = cur.execute(sql)
    return(r.fetchall())

In [225]:
# to show all rows and columns 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [226]:
test = pd.DataFrame(run("""SELECT 
    * 
FROM  Purchase P
LEFT JOIN Cart C ON P.Cart_ID = C.ID
LEFT JOIN Product PR ON C.Product_ID = PR.ID
LEFT JOIN Category CY ON PR.Category_ID = CY.ID
ORDER BY row_ID DESC"""))

# Shoud be 51290 items


In [227]:
display(test.tail())
test.shape


,Customer_ID,Order_Date,Cart_ID,Address_ID,Order_TwoLC,Ship_Date,Ship_Mode,Order_Priority,Returned,ID,Row_ID,Product_ID,Quantity,Sales,Discount,Profit,Shipping_Cost,Product_Name,Category_ID,Unit_Price,Category,SubCategory
51285,439,2015-10-15 00:00:00,8384,844,MX,2015-10-20 00:00:00,Standard Class,Medium,0,5905,5,4745,2,71.60,0.0,11.44,3.787,"Sanford Canvas, Water Color",5905,35.80,Office Supplies,Art
51286,439,2015-10-15 00:00:00,8384,844,MX,2015-10-20 00:00:00,Standard Class,Medium,0,3720,4,1362,4,35.44,0.0,4.96,1.371,"Cardinal Binder, Clear",3720,8.86,Office Supplies,Binders
51287,439,2015-10-15 00:00:00,8384,844,MX,2015-10-20 00:00:00,Standard Class,Medium,0,4845,3,3122,2,193.28,0.0,54.08,9.627,"Ikea 3-Shelf Cabinet, Mobile",4845,96.64,Furniture,Bookcases
51288,439,2015-10-15 00:00:00,8384,844,MX,2015-10-20 00:00:00,Standard Class,Medium,0,6238,2,5385,8,252.16,0.0,90.72,13.449,"Tenex Clock, Durable",6238,31.52,Furniture,Furnishings
51289,706,2017-10-02 00:00:00,22814,1880,MX,2017-10-06 00:00:00,Standard Class,Medium,0,4658,1,2821,3,13.08,0.0,4.56,2.033,"Hon File Folder Labels, Adjustable",4658,4.36,Office Supplies,Labels


(51290, 22)

# Tasks
* What is the category generating the maximum sales revenue?
  * What about the profit in this category?
  * Are they making a loss in any categroies?
* What are 5 states generating the maximum and minimum sales revenue?
* What are the 3 products in each product segment with the highest sales?
  * Are they the 3 most profitable products as well?
* What are the 3 best-seller products in each product segment? (Quantity-wise)
* What are the top 3 worst-selling products in every category? (Quantity-wise)
* How many unique customers per month are there for the year 2016. 
`(There's a catch here: contrary to other 'heavier' RDBMS, SQLite does not support the functions YEAR() or MONTH() to extract the year or the month in a date. You will have to create two new columns: year and month.)`

## What is the category generating the maximum sales revenue?

In [228]:
display(pd.DataFrame(run("""SELECT 
    Category,
    sum(Sales) as TotalSales 
FROM  Purchase P
LEFT JOIN Cart C ON P.Cart_ID = C.ID
LEFT JOIN Product PR ON C.Product_ID = PR.ID
LEFT JOIN Category CY ON PR.Category_ID = CY.ID
LEFT JOIN Address A ON P.Address_ID = A.ID
LEFT JOIN City CI ON A.City_ID = CI.ID
LEFT JOIN Country CO ON CI.Country_ID = CO.ID
WHERE CO.Country = "United States"
GROUP BY Category
ORDER BY TotalSales DESC
LIMIT 5
""")))

,Category,TotalSales
0,Technology,836154.0330
1,Furniture,741577.5073
2,Office Supplies,719469.3200


### What about the profit in this category?
### Are they making a loss in any categroies?

In [229]:
display(pd.DataFrame(run("""SELECT 
    Category,
    sum(Profit) AS SumProfit,
    CASE WHEN Sum(Profit) < 0
       THEN 'Yes'
       ELSE 'No'
       END AS MakingLoss
FROM  Purchase P
LEFT JOIN Cart C ON P.Cart_ID = C.ID
LEFT JOIN Product PR ON C.Product_ID = PR.ID
LEFT JOIN Category CY ON PR.Category_ID = CY.ID
LEFT JOIN Address A ON P.Address_ID = A.ID
LEFT JOIN City CI ON A.City_ID = CI.ID
LEFT JOIN Country CO ON CI.Country_ID = CO.ID
WHERE CO.Country = "United States"
GROUP BY Category
ORDER BY SumProfit DESC
LIMIT 5
""")))

,Category,SumProfit,MakingLoss
0,Technology,145454.9481,No
1,Office Supplies,122612.7944,No
2,Furniture,18329.2792,No


## What are 5 states generating the maximum and minimum sales revenue?

In [230]:
display(pd.DataFrame(run("""select * from ( SELECT 
    State,
    sum(Sales) as TotalSales 
FROM  Purchase P
LEFT JOIN Cart C ON P.Cart_ID = C.ID
LEFT JOIN Product PR ON C.Product_ID = PR.ID
LEFT JOIN Category CY ON PR.Category_ID = CY.ID
LEFT JOIN Address A ON P.Address_ID = A.ID
LEFT JOIN City CI ON A.City_ID = CI.ID
LEFT JOIN Country CO ON CI.Country_ID = CO.ID
WHERE CO.Country = "United States"
GROUP BY State
ORDER BY TotalSales DESC
LIMIT 5 )

UNION 

select * from ( SELECT 
    State,
    sum(Sales) as TotalSales 
FROM  Purchase P
LEFT JOIN Cart C ON P.Cart_ID = C.ID
LEFT JOIN Product PR ON C.Product_ID = PR.ID
LEFT JOIN Category CY ON PR.Category_ID = CY.ID
LEFT JOIN Address A ON P.Address_ID = A.ID
LEFT JOIN City CI ON A.City_ID = CI.ID
LEFT JOIN Country CO ON CI.Country_ID = CO.ID
WHERE CO.Country = "United States"
GROUP BY State
ORDER BY TotalSales ASC
LIMIT 5 )
ORDER BY TotalSales ASC
""")))

,State,TotalSales
0,North Dakota,919.9100
1,West Virginia,1209.8240
2,Maine,1270.5300
3,South Dakota,1315.5600
4,Wyoming,1603.1360
5,Pennsylvania,116511.9140
6,Washington,138641.2700
7,Texas,170188.0458
8,New York,310876.2710
9,California,457687.6315


In [214]:
# to show all rows and columns 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [221]:
display(pd.DataFrame(run("""SELECT 
    Cart_ID,
    GROUP_CONCAT(row_id),
    count(*) as c,
    sum(Sales) as TotalSales
FROM  Purchase P
LEFT JOIN Cart C ON P.Cart_ID = C.ID
LEFT JOIN Address A ON P.Address_ID = A.ID
LEFT JOIN City CI ON A.City_ID = CI.ID
LEFT JOIN Country CO ON CI.Country_ID = CO.ID
WHERE CO.Country = "United States"
  AND State = "California"
GROUP BY State
HAVING c > 1""")))

,Cart_ID,GROUP_CONCAT(row_id),c,TotalSales
0,1073,"36651,36652,37425,40976,40977,40834,34336,3629...",2212,509384.4675


## What are the 3 products in each product segment with the highest sales

### Are they the 3 most profitable products as well?

In [163]:
display(pd.DataFrame(run("""
SELECT Category, Product_Name, TotalSales, CASE WHEN rn_p <= 3
       THEN 'Yes'
       ELSE 'No'
       END AS AlsoMostProfitable FROM (
SELECT  
    ROW_NUMBER() OVER (PARTITION BY Category ORDER BY TotalSales DESC) as rn,
    ROW_NUMBER() OVER (PARTITION BY Category ORDER BY AvgProfit DESC) as rn_p,
    *
FROM (
SELECT
    Category,
    Product_Name,
    sum(Sales) AS TotalSales,
    avg(Profit) As AvgProfit
FROM Purchase P
LEFT JOIN Cart C ON P.Cart_ID = C.ID
LEFT JOIN Product PR ON C.Product_ID = PR.ID
LEFT JOIN Category CY ON PR.Category_ID = CY.ID
LEFT JOIN Address A ON P.Address_ID = A.ID
LEFT JOIN City CI ON A.City_ID = CI.ID
LEFT JOIN Country CO ON CI.Country_ID = CO.ID
LEFT JOIN Customer CU ON P.Customer_ID = CU.ID
WHERE CO.Country = "United States"
GROUP BY Segment, Product_Name
) )
WHERE rn <= 3
ORDER BY Category, TotalSales DESC
""")))

,Category,Product_Name,TotalSales,AlsoMostProfitable
0,Furniture,HON 5400 Series Task Chairs for Big and Tall,17384.3040,No
1,Furniture,DMI Eclipse Executive Suite Bookcases,11046.6090,No
2,Furniture,"Riverside Palais Royal Lawyers Bookcase, Royal...",9708.3996,No
3,Office Supplies,GBC Ibimaster 500 Manual ProClick Binding System,17122.0500,No
4,Office Supplies,Fellowes PB500 Electric Punch Plastic Comb Bin...,14489.2860,No
5,Office Supplies,GBC DocuBind TL300 Electric Binding System,13096.0540,No
6,Technology,Canon imageCLASS 2200 Advanced Copier,32899.9060,Yes
7,Technology,Cisco TelePresence System EX90 Videoconferenci...,22638.4800,No
8,Technology,HP Designjet T520 Inkjet Large Format Printer ...,18374.8950,No


## What are the 3 best-seller products in each product segment? (Quantity-wise)

In [164]:
display(pd.DataFrame(run("""
SELECT Category, Product_Name, TotalSales FROM (
SELECT ROW_NUMBER() OVER (PARTITION BY Category ORDER BY TotalSales DESC) as rn, * FROM (
SELECT
    Category,
    Product_Name,
    count(*) AS TotalSales 
FROM Purchase P
LEFT JOIN Cart C ON P.Cart_ID = C.ID
LEFT JOIN Product PR ON C.Product_ID = PR.ID
LEFT JOIN Category CY ON PR.Category_ID = CY.ID
LEFT JOIN Address A ON P.Address_ID = A.ID
LEFT JOIN City CI ON A.City_ID = CI.ID
LEFT JOIN Country CO ON CI.Country_ID = CO.ID
LEFT JOIN Customer CU ON P.Customer_ID = CU.ID
WHERE CO.Country = "United States"
GROUP BY Category, Product_Name

) )
WHERE rn <= 3
ORDER BY Category, TotalSales DESC
""")))

,Category,Product_Name,TotalSales
0,Furniture,KI Adjustable-Height Table,18
1,Furniture,"Situations Contoured Folding Chairs, 4/Set",15
2,Furniture,Eldon Wave Desk Accessories,14
3,Office Supplies,Staples,227
4,Office Supplies,Avery Non-Stick Binders,20
5,Office Supplies,Storex Dura Pro Binders,17
6,Technology,Logitech 910-002974 M325 Wireless Mouse for We...,15
7,Technology,Kingston Digital DataTraveler 16GB USB 2.0,13
8,Technology,Logitech Desktop MK120 Mouse and keyboard Combo,12


## What are the top 3 worst-selling products in every category? (Quantity-wise)

In [154]:
display(pd.DataFrame(run("""
SELECT Category, Product_Name, sells, AvgProfit FROM (
SELECT *,
    ROW_NUMBER() OVER (PARTITION BY Category ORDER BY sells, AvgProfit ASC) as rn FROM (
SELECT 
    Category,
    Product_Name,
    avg(Profit) AS AvgProfit,
    count(*) AS sells
FROM  Purchase P
LEFT JOIN Cart C ON P.Cart_ID = C.ID
LEFT JOIN Product PR ON C.Product_ID = PR.ID
LEFT JOIN Category CY ON PR.Category_ID = CY.ID
LEFT JOIN Address A ON P.Address_ID = A.ID
LEFT JOIN City CI ON A.City_ID = CI.ID
LEFT JOIN Country CO ON CI.Country_ID = CO.ID
WHERE CO.Country = "United States"
GROUP BY Category, Product_Name) )
WHERE rn <= 3
""")))

,Category,Product_Name,sells,AvgProfit
0,Furniture,"Bush Westfield Collection Bookcases, Dark Cher...",1,-190.8522
1,Furniture,"Bush Birmingham Collection Bookcase, Dark Cherry",1,-117.8820
2,Furniture,"Bush Saratoga Collection 5-Shelf Bookcase, Han...",1,-12.6882
3,Office Supplies,Eureka Disposable Bags for Sanitaire Vibra Gro...,1,-4.4660
4,Office Supplies,Sanitaire Vibra Groomer IR Commercial Upright ...,1,1.3000
5,Office Supplies,Grip Seal Envelopes,1,2.3868
6,Technology,Cubify CubeX 3D Printer Triple Head Print,1,-3839.9904
7,Technology,Cisco TelePresence System EX90 Videoconferenci...,1,-1811.0784
8,Technology,Zebra GK420t Direct Thermal/Thermal Transfer P...,1,-938.2800


## How many unique customers per month are there for the year 2016. 

In [162]:
display(pd.DataFrame(run("""
SELECT strftime('%m', Order_Date) AS Month,
       COUNT(DISTINCT customer_ID) AS UniqueCustomers
FROM Purchase P
LEFT JOIN Address A ON P.Address_ID = A.ID
LEFT JOIN City CI ON A.City_ID = CI.ID
LEFT JOIN Country CO ON CI.Country_ID = CO.ID
WHERE CO.Country = "United States"
AND strftime('%Y', Order_Date) = '2016'
GROUP BY Month;
""")))

,Month,UniqueCustomers
0,01,46
1,02,41
2,03,78
3,04,82
4,05,97
5,06,90
6,07,89
7,08,86
8,09,174
9,10,92


# DB Disconnect

In [ ]:
cur.close()
con.close()